In [1]:
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime
import networkx as nx
from ripser import ripser
from persim import PersistenceImager
import matplotlib.pyplot as plt
import time


# sample.csvのあるディレクトリを指定
os.chdir("/Users/sinco/Desktop/python")

# CSVファイルの読み取り
df = pd.read_csv('2018-transaction-3.txt')  # トランザクションデータ
df_phish = pd.read_csv('phish.csv')  # フィッシングアカウント一覧

# フィッシングアカウントのリストを取得
phish_list = df_phish["account"].values.tolist()
df['is_phish'] = df['from_address'].isin(phish_list) | df['to_address'].isin(phish_list)
df_ph = df[df['is_phish']]  # フィッシングアカウントに関連するトランザクションを抽出

# 新しいフォルダを作成 (日時で一意に)
folder_name = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_path = f"/Users/sinco/Desktop/python/pimgs/{folder_name}/"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# ノーマルアカウントのデータを抽出
df_normal = df[~df['is_phish']]  # フィッシングアカウントでないトランザクションを抽出

# 抽出する行数と割合を指定
num_samples = 5700  # 抽出する総行数
phish_ratio = 0  # フィッシングアカウントの割合 (0.3 = 30%)

# フィッシングとノーマルの行数を計算
num_phish_samples = int(num_samples * phish_ratio)
num_normal_samples = num_samples - num_phish_samples

# フィッシングアカウントとノーマルアカウントをランダムにサンプリング
df_ph_sample = df_ph.sample(n=min(num_phish_samples, len(df_ph)), random_state=42)  # フィッシングアカウント
df_normal_sample = df_normal.sample(n=min(num_normal_samples, len(df_normal)), random_state=42)  # ノーマルアカウント

# サンプルを結合
df_sample = pd.concat([df_ph_sample, df_normal_sample])

# グラフを作成
edges = pd.DataFrame({
    'source': df_sample['from_address'],
    'target': df_sample['to_address'],
    'weight': df_sample['value']
})
G = nx.from_pandas_edgelist(edges, edge_attr=True)

# 1000枚のグラフを作成
for i in range(300):
   
    #plt.figure(figsize=(28/100, 28/100))  # グラフのサイズを28x28に設定（縮小）
    nx.draw(
        G,
        with_labels=False,  # ラベルを非表示にする
        node_size=0.12,       # ノードのサイズを小さく
        width=0.07   # エッジの線の太さを細く設定
    )
    
    # 画像ファイルとして保存
    img_path = f"{folder_path}graph_{i+1}.png"
    plt.savefig(img_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 
    plt.close()  # プロットを閉じてメモリ解放
